To use the project, make sure to run the first three sets of code each time you open the colab. 

It is also important to run the code block that defines get_frame_number(). You can find it under a big text block that has a long blue URL at the end.

For adding to the 'left','right'and 'null' directories, look for the text blocks that talk about adding 100 frames to those directories, and run those 3 snippets.

#This mounts the files from your google drive into the following directory

content->drive->MyDrive->CEN5727 - HCI Project

In [ ]:
from google.colab import drive
drive.mount('/SharedDrives')

Mounted at /SharedDrives


#This next line will put you in the root of our project folder

In [ ]:
%cd /SharedDrives/MyDrive/Colab Notebooks

/SharedDrives/.shortcut-targets-by-id/1rb6ApsTN377Q22zZoxKBzvRlXNcjYUFp/Colab Notebooks


#List of libraries used. Chunks include the needed libraries in each respective chunk.

In [ ]:
import os
import os.path
import cv2
import time
import shutil
import glob
import random
import numpy as np
import h5py

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from distutils.dir_util import copy_tree
from keras.preprocessing import image
from keras.models import Sequential
from keras.models import load_model

#Include all dependancies here. Only need to be run once and google collab will maintain the installation.

In [ ]:
!pip install opencv-python
!pip install keras
!pip install tensorflow

#Making the directories where the images will be stored

In [ ]:
import os

directories = ['right', 'left', 'null'] #3 classifications initially

parent_dir = os.getcwd() #Current directory
print(parent_dir)  

#
for i in range(3):
    path = os.path.join(parent_dir, directories[i])
    try: 
        os.mkdir(path)
    except OSError as error:
        print(error)
        
    print("Directory '% s' created" % directories[i])

#randomizer from kev


In [ ]:
#Little script that checks last value and creates variable called "last number.""
import os,os.path

null_frame_number=0
left_frame_number=0
rigth_frame_number=0


def get_frame_numbers():
  #we will still need to add +1 to these variables later in the code
  #but this function should count how many files are in a folder, and that number
  #would be exactly what the last frame was, ie it would be 99 if the hightest 
  #frame number in
  global left_frame_number
  global right_frame_number
  global null_frame_number
  %cd /SharedDrives/MyDrive/Colab\ Notebooks/left
  %pwd #print workingdirectory
  left_frame_number = (len([name for name in os.listdir('.') if os.path.isfile(name)])) #line from stack overflow
  print ("^LEFT has this many frames: ", left_frame_number)
  %cd /SharedDrives/MyDrive/Colab\ Notebooks/right
  %pwd #print workingdirectory
  right_frame_number = (len([name for name in os.listdir('.') if os.path.isfile(name)])) #line from stack overflow
  print ("^RIGHT has this many frames: ", right_frame_number)
  %cd /SharedDrives/MyDrive/Colab\ Notebooks/null
  %pwd #print workingdirectory
  null_frame_number = (len([name for name in os.listdir('.') if os.path.isfile(name)])) #line from stack overflow
  print ("^NULL has this many frames: ", null_frame_number)
  #now we need to return to our previous working directory, sort of a 'clean-up' step
  %cd /SharedDrives/MyDrive/Colab\ Notebooks/

#test the function
get_frame_numbers()


#Testing camera capture feature of google collab. 
https://stackoverflow.com/questions/54389727/opening-web-camera-in-google-colab

#Run this cell to enable the google colab camera API.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

#The snippet below will generate a camera frame and allow you to take a picture.

#Next step is saving that picture to the relevant class.

In [ ]:
import os

os.chdir("/SharedDrives/MyDrive/Colab Notebooks")

from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

#Portion below captures frame every second for 100 seconds and saves to 'right' directory. CAN ONLY BE RUN LOCALLY.

In [ ]:
import os
import cv2
import time

parent_dir = os.getcwd() #Current directory
directories = ['right', 'left', 'null'] #Classifications
path = os.path.join(parent_dir, directories[0]) 

#vid = cv2.VideoCapture(0, cv2.CAP_DSHOW) #Opens the inbuilt camera, trying to remove delay with second argument
# or uncomment this code to read from a file instead
vid = cv2.VideoCapture("kev_prerecorded/right1.mp4") #this can be commented in favour of using camera over prerecorded

#first we need to figure out where we are in the directory
get_frame_numbers()
i = right_frame_number + 1 #add all the frames together, add 1 to start on the next frame

time.sleep(5) #5 second pause to get in position

while(True):
    ret, frame = vid.read() #Capture frame by frame
    #cv2.imshow('frame', frame) #Display on frame, temporarily commented this just to get the code to run inside of colab
    
    if cv2.waitKey(1) & 0xFF == ord('q'): #Press 'q' to stop
        break
    
    #Saving frames to 'right' folder
    cv2.imwrite(os.path.join(path, 'Frame'+str(i)+'.jpg'), frame)
    i +=1
    time.sleep(1) #Every 1 second
    if (i == 240 + right_frame_number + 1): #this should make 240 frames, cuz we have 240 + wherever our last frame was + 1, where 1 represents where we are starting at
        break
            
vid.release()
cv2.destroyAllWindows()
get_frame_numbers() #run it again just to make sure nothing is accidentally overwritten

In [ ]:
import os
import cv2
import time

parent_dir = os.getcwd() #Current directory
directories = ['right', 'left', 'null'] #Classifications
path = os.path.join(parent_dir, directories[1]) 

vid = cv2.VideoCapture(0, cv2.CAP_DSHOW) #Opens the inbuilt camera, trying to remove delay with second argument

i = left_frame_number + 1 #count all frames in "left", add 1

time.sleep(5) #5 second pause to get in position

while(True):
    ret, frame = vid.read() #Capture frame by frame
    cv2.imshow('frame', frame) #Display on frame
    
    if cv2.waitKey(1) & 0xFF == ord('q'): #Press 'q' to stop
        break
    
    #Saving frames to 'right' folder
    cv2.imwrite(os.path.join(path, 'Frame'+str(i)+'.jpg'), frame)
    i +=1
    time.sleep(1) #Every 1 second
    if (i == 100):
        break
            
vid.release()
cv2.destroyAllWindows()

https://stackoverflow.com/questions/18805348/how-rename-the-images-in-folder

#Code below looks in the class directory within "parent_dir" and randomizes all the images moving them into a random folder

In [ ]:
os.chdir("/SharedDrives/MyDrive/Colab Notebooks")
parent_dir = os.getcwd()

class_dir = ['right', 'left', 'null']
class_dir_destination = ['right_random', 'left_random', 'null_random']

for i in range(3):
  current_dir = os.path.join(parent_dir, class_dir[i]) #Changing into class folder
  dest_dir = os.path.join(parent_dir, class_dir_destination[i])
  count = 0 #This will just change the picture to 0 through the number of images
  os.chdir(current_dir) 
  for jpgfile in glob.iglob(os.path.join(current_dir, "*.jpg")): #Iterating through the files
    random_image = random.choice(os.listdir(current_dir)) #Any file in class folder
    old_name = os.path.join(current_dir, random_image)
    new_name = os.path.join(dest_dir, str(count)+ ".jpg")
    #print("Old name: ", old_name)
    #print("New name: ", new_name)
    os.rename(old_name, new_name)
    #print(old_name + " ---> " + new_name)
    count += 1


#Moves back from random into left, null, right 


In [ ]:
os.chdir("/SharedDrives/MyDrive/Colab Notebooks")
parent_dir = os.getcwd()

class_dir_destination = ['right', 'left', 'null']
class_dir = ['right_random', 'left_random', 'null_random']

for i in range(3):
  current_dir = os.path.join(parent_dir, class_dir[i]) #Changing into class folder
  dest_dir = os.path.join(parent_dir, class_dir_destination[i])
  os.chdir(current_dir) 
  for file_name in os.listdir(current_dir):
    #random_image = random.choice(os.listdir(current_dir)) #Any file in class folder
    source = os.path.join(current_dir, file_name)
    destination = os.path.join(dest_dir, file_name)
    if os.path.isfile(source):
      shutil.move(source, destination)
      print('Moved', file_name, ' from', source, ' to', destination)


#Function below counts all the images for the classes in the specified folder and returns the numbers in the "right" "left" "null" order.

In [ ]:
def class_count(src):
  class_dir = ['right', 'left', 'null'] #Classifications
  os.chdir(src)
  parent_dir = os.getcwd()
  #parent_dir = os.chdir(os.path.join(parent_dir, "training_set"))

  data_counts = [0, 0, 0]

  count_val = 0 

  for i in range(3): 
      count_val = 0
      src_dir = os.path.join(parent_dir, class_dir[i])
      for jpgfile in glob.iglob(os.path.join(src_dir,"*.jpg")):
          count_val += 1
          
      data_counts[i] = count_val

  return data_counts

#Getting count in each folder

In [ ]:
#Counting "right" "left" "null" in /Colab Notebooks
root_rln_count = class_count("/SharedDrives/MyDrive/Colab Notebooks")
print("In /Colab Notebooks, 'right'  'left'  'null:", root_rln_count)

#Counting "right" "left" "null" in /Colab Notebooks/training_set
training_rln_count = class_count("/SharedDrives/MyDrive/Colab Notebooks/training_set")
print("In /Colab Notebooks/training_set, 'right'  'left'  'null:", training_rln_count)

#Counting "right" "left" "null" in /Colab Notebooks/test_set
test_rln_count = class_count("/SharedDrives/MyDrive/Colab Notebooks/test_set")
print("In /Colab Notebooks/test_set, 'right'  'left'  'null:", test_rln_count)


In /Colab Notebooks, 'right'  'left'  'null: [636, 636, 636]
In /Colab Notebooks/training_set, 'right'  'left'  'null: [605, 605, 605]
In /Colab Notebooks/test_set, 'right'  'left'  'null: [31, 31, 31]


#The bit below counts the number of files that will be training and test based on the percentage selected. 

In [ ]:
import numpy as np

train_per = input("What percentage to make your training set: ")

train_counts = np.array(root_rln_count) * (int(train_per)/100)

train_counts = np.ceil(train_counts)

test_counts = np.ceil(root_rln_count - np.ceil(train_counts))

print(train_counts)
print(test_counts)

What percentage to make your training set: 90
[573. 573. 573.]
[63. 63. 63.]


#Function below copies from the chose directory "parent_dir" into training and test based on the amount selected "train_counts" and "test_counts"

In [ ]:
def train_test_splitter(parent_dir, train_counts, test_counts):

  #parent_dir = ("/SharedDrives/MyDrive/Colab Notebooks/testy")

  os.chdir(parent_dir)
  os.getcwd()

  #training_dir = ("/SharedDrives/MyDrive/Colab Notebooks/testy/training_set")
  #test_dir = ("/SharedDrives/MyDrive/Colab Notebooks/testy/test_set")

  training_dir = os.path.join(parent_dir, "training_set")
  test_dir = os.path.join(parent_dir, "test_set")

  class_dir = ['right', 'left', 'null']

  for i in range(3):
    count = 0
    current_dir = os.path.join(parent_dir, class_dir[i]) #Changing into class folder

    for file_name in os.listdir(current_dir):
      if (count < train_counts[i]):
        dest_dir = training_dir
      else:
        dest_dir = test_dir

      #Remove the line below if it doesn't work
      #random_image = random.choice(os.listdir(current_dir)) #Select random image

      #Change 'random_image' back to 'file_name' if it doesn't work
      source = os.path.join(current_dir, file_name)
      destination = os.path.join(dest_dir, class_dir[i])

      if os.path.isfile(source):
        shutil.copy(source, destination)
        print('copied:', file_name, ' from', source, ' to ', destination)
        count +=1

#Dividing the dataset into training and test


In [ ]:
train_test_splitter("/SharedDrives/MyDrive/Colab Notebooks", train_counts, test_counts)

#Deleting all files in test_set and training_set of root

In [ ]:
def remove_images(directory_path):
  for f in os.listdir(directory_path):
    os.remove(os.path.join(directory_path,f))

In [ ]:
remove_images("/SharedDrives/MyDrive/Colab Notebooks/training_set/right")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/training_set/left")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/training_set/null")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/test_set/right")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/test_set/left")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/test_set/null")

In [ ]:
remove_images("/SharedDrives/MyDrive/Colab Notebooks/left")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/right")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/null")

In [ ]:
remove_images("/SharedDrives/MyDrive/Colab Notebooks/left_random")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/right_random")
remove_images("/SharedDrives/MyDrive/Colab Notebooks/null_random")

#Back up class folders from root folder into DataBackup

In [ ]:
def image_backup(parent_dir, dest_dir):
  
  """
  parent_dir = "/SharedDrives/MyDrive/Colab Notebooks/"
  dest_dir = "/SharedDrives/MyDrive/Colab Notebooks/DataBackup"
  """
  
  class_dir = ['right', 'left', 'null']

  for i in range(3):
    current_dir = os.path.join(parent_dir, class_dir[i]) #Changing into class folder

    for file_name in os.listdir(current_dir):
      source = os.path.join(current_dir, file_name)
      destination = os.path.join(dest_dir, class_dir[i])
      if os.path.isfile(source):
        shutil.copy(source, destination)
        print('copied: ', file_name, ' from ', source, ' to', destination)

In [ ]:
#Copying from DataBackup to root
image_backup("/SharedDrives/MyDrive/Colab Notebooks/DataBackup", "/SharedDrives/MyDrive/Colab Notebooks")

In [ ]:
#Copying from root to DataBackup
image_backup("/SharedDrives/MyDrive/Colab Notebooks", "/SharedDrives/MyDrive/Colab Notebooks/DataBackup")

#Code below copied data from root folder the class folders into testy data folders

In [ ]:
import os
from distutils.dir_util import copy_tree

os.chdir("/SharedDrives/MyDrive/Colab Notebooks")
print(os.getcwd())
class_dir = ['right', 'left', 'null'] #Classifications

source_folder = "/SharedDrives/MyDrive/Colab Notebooks"
destination_folder = "/SharedDrives/MyDrive/Colab Notebooks/testy"

# fetch all files
for i in range(3):
    # construct full file path
    source = str(source_folder +"/" + class_dir[i])
    print(source)
    destination = str(destination_folder + "/" + class_dir[i])
    print(destination)
    copy_tree(source, destination)

In [ ]:
import glob
import shutil
import os

class_dir = ['right', 'left', 'null'] #Classifications
os.chdir("/SharedDrives/MyDrive/Colab Notebooks")
parent_dir = os.getcwd()

total_data_counts = [0, 0, 0]

count_val = 0 

for i in range(3): 
    count_val = 0
    src_dir = os.path.join(parent_dir, class_dir[i])
    for jpgfile in glob.iglob(os.path.join(src_dir,"*.jpg")):
        count_val += 1
        
    total_data_counts[i] = count_val

total_data_counts

[339, 339, 339]

#Count "left_random", "right_random", etc.

In [ ]:
import glob
import shutil
import os

class_dir = ['right_random', 'left_random', 'null_random'] #Classifications
os.chdir("/SharedDrives/MyDrive/Colab Notebooks/testy")
parent_dir = os.getcwd()

total_data_counts = [0, 0, 0]

count_val = 0 

for i in range(3): 
    count_val = 0
    src_dir = os.path.join(parent_dir, class_dir[i])
    for jpgfile in glob.iglob(os.path.join(src_dir,"*.jpg")):
        count_val += 1
        
    total_data_counts[i] = count_val

total_data_counts

[0, 0, 0]

#Counting training in /testy


In [ ]:
import glob
import shutil
import os

class_dir = ['right', 'left', 'null'] #Classifications
os.chdir(r"/SharedDrives/MyDrive/Colab Notebooks/testy/training_set")
parent_dir = os.getcwd()
#parent_dir = os.chdir(os.path.join(parent_dir, "training_set"))

training_data_counts = [0, 0, 0]

count_val = 0 

for i in range(3): 
    count_val = 0
    src_dir = os.path.join(parent_dir, class_dir[i])
    for jpgfile in glob.iglob(os.path.join(src_dir,"*.jpg")):
        count_val += 1
        
    training_data_counts[i] = count_val

training_data_counts

[306, 306, 306]

#Counting test in /testy

In [ ]:
import glob
import shutil
import os


class_dir = ['right', 'left', 'null'] #Classifications
os.chdir(r"/SharedDrives/MyDrive/Colab Notebooks/testy/test_set")
parent_dir = os.getcwd()
#parent_dir = os.chdir(os.path.join(parent_dir, "training_set"))

test_data_counts = [0, 0, 0]

count_val = 0 

for i in range(3): 
    count_val = 0
    src_dir = os.path.join(parent_dir, class_dir[i])
    for jpgfile in glob.iglob(os.path.join(src_dir,"*.jpg")):
        count_val += 1
        
    test_data_counts[i] = count_val

test_data_counts
#model_test_counts = sum(test_data_counts)
#print(model_test_counts)

[33, 33, 33]

#Moving from random class folders into training and test according to the percentages

In [ ]:
#USE train_counts AND test_counts

parent_dir = ("/SharedDrives/MyDrive/Colab Notebooks")

os.chdir(parent_dir)
os.getcwd()

#training_dir = ("/SharedDrives/MyDrive/Colab Notebooks/testy/training_set")
#test_dir = ("/SharedDrives/MyDrive/Colab Notebooks/testy/test_set")

class_dir = ['right', 'left', 'null']

for i in range(3):
  count = 0
  current_dir = os.path.join(parent_dir, class_dir[i]) #Changing into class folder

  for file_name in os.listdir(current_dir):
    if (count < train_counts[1]):
      dest_dir = training_dir
    else:
      dest_dir = test_dir

    random_image = random.choice(os.listdir(current_dir)) #Any file in class folder

    source = os.path.join(current_dir, random_image)
    destination_inter = os.path.join(dest_dir, class_dir[i])
    destination = os.path.join(destination_inter, random_image)
    if os.path.isfile(source):
      shutil.move(source, destination)
      print('Moved', random_image)
      count +=1

FileNotFoundError: ignored

In [ ]:
#Just to remove folders with contents
"""
!rm -rf "/SharedDrives/MyDrive/Colab Notebooks/testy/training_set/right"
!rm -rf "/SharedDrives/MyDrive/Colab Notebooks/testy/training_set/left"
!rm -rf "/SharedDrives/MyDrive/Colab Notebooks/testy/training_set/null"

!rm -rf "/SharedDrives/MyDrive/Colab Notebooks/DataBackup"
"""


In [ ]:
#Function to copy all contents of source "src" into destination "drc"
def copying_folder(src, drc):
  from pathlib import Path
  import shutil
 
  files=os.listdir(src)
 
  # iterating over all the files in
  # the source directory
  for fname in files:
     
    # copying the files to the
    # destination directory
    shutil.copy2(os.path.join(src,fname), drc)

In [ ]:
copying_folder("/SharedDrives/MyDrive/Colab Notebooks/left", "/SharedDrives/MyDrive/Colab Notebooks/DataBackup/left")
copying_folder("/SharedDrives/MyDrive/Colab Notebooks/right", "/SharedDrives/MyDrive/Colab Notebooks/DataBackup/right")
copying_folder("/SharedDrives/MyDrive/Colab Notebooks/null", "/SharedDrives/MyDrive/Colab Notebooks/DataBackup/null")


#Based on yoga-pose-CNN by Anastasia Merchenkova
#A CNN in Keras
https://github.com/amarchenkova/yoga-pose-CNN/blob/master/cnn_multiclass.py

In [ ]:
%run -i "cnn_multiclass.py"

Found 210 images belonging to 3 classes.
Found 87 images belonging to 3 classes.


C:\Users\deoli\CEN5728 - HCI\cnn_multiclass.py:45: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_set,


6/6 [==============================] - 3s 465ms/step - loss: 1.2102 - accuracy: 0.3315 - val_loss: 1.0962 - val_accuracy: 0.3281


#The chunk below is the text from the "cnn_multiclass.py" file. You can run it either way. From the run command above, or the chunk below.

###VERY IMPORTANT: Variable "steps_per_epoch" and "validation_steps" must be change according to the total amount of training and test (respectivelY) data you have. 

In [ ]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display 
from PIL import Image
import h5py

os.chdir("/SharedDrives/MyDrive/Colab Notebooks") #UNCOMMENT AFTER TESTING
#os.chdir("/SharedDrives/MyDrive/Colab Notebooks/testy") #REMOVE AFTER TESTING THIS SHOULD RUN IN BASE DIRECTORY

# CNN model 
classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax')) # number of classes, units changed from 10 to 3

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Data Augmentation
batch_size = 32
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 20,
                                   horizontal_flip = False) #Changed from true to false. I do not want it to flip since that's a different command

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (64, 64),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('test_set',
                                            target_size = (64, 64),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

#training_rln_count 
#test_rln_count
classifier.fit_generator(training_set,
                         steps_per_epoch = sum(training_rln_count) // batch_size, # number of training set images, 729 SWITCHED
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = sum(test_rln_count) // batch_size) # number of test set images, 229 SWITCHED


classifier.save('imageClassifier10.h5') #save model 
classifier.summary()

Found 1815 images belonging to 3 classes.
Found 93 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
56/56 [==============================] - 46s 815ms/step - loss: 0.8919 - accuracy: 0.5799 - val_loss: 0.3773 - val_accuracy: 0.8750
Epoch 2/5
56/56 [==============================] - 43s 768ms/step - loss: 0.3831 - accuracy: 0.8536 - val_loss: 0.1399 - val_accuracy: 0.9688
Epoch 3/5
56/56 [==============================] - 43s 759ms/step - loss: 0.1924 - accuracy: 0.9310 - val_loss: 0.0385 - val_accuracy: 1.0000
Epoch 4/5
56/56 [==============================] - 43s 763ms/step - loss: 0.1207 - accuracy: 0.9619 - val_loss: 0.0223 - val_accuracy: 1.0000
Epoch 5/5
56/56 [==============================] - 43s 762ms/step - loss: 0.1230 - accuracy: 0.9568 - val_loss: 0.0460 - val_accuracy: 1.0000
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max

#Test your model using the script below. Change the load_model() parameter to the latest model and select your image in the load_img() parameter

In [ ]:
"""
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.models import load_model
import h5py
import os
"""

os.chdir("/SharedDrives/MyDrive/Colab Notebooks") 

classifier = load_model('imageClassifier10.h5') #load the model that was created using cnn_multiclass.py

test_image = image.load_img('left/260.jpg', target_size = (64, 64)) #folder predictions with images that I want to test
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
 
result = classifier.predict(test_image) # returns array

if result[0][0] == 1:
	prediction = 'left' #predictions in array are in alphabetical order
elif result[0][1] == 1:
	prediction = 'null'
elif result[0][2] == 1:
	prediction = 'right'

print(result)
print(prediction)

[[1. 0. 0.]]
left


In [ ]:
from IPython.display import Image
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.models import load_model
import h5py

os.chdir("/SharedDrives/MyDrive/Colab Notebooks")

try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))


classifier = load_model('imageClassifier10.h5') #load the model that was created using cnn_multiclass.py

test_image = image.load_img('photo.jpg', target_size = (64, 64)) #folder predictions with images that I want to test
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
 
result = classifier.predict(test_image) # returns array

if result[0][0] == 1:
	prediction = 'left' #predictions in array are in alphabetical order
elif result[0][1] == 1:
	prediction = 'null'
elif result[0][2] == 1:
	prediction = 'right'

print(result)
print(prediction)



#Later in the project, we will use local runtimes to connect the photo to a local host so that the pyautogui functions can be used and programmatic control achieved. 